In [30]:
import cv2
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation
from keras.utils import to_categorical


In [7]:
INPUT_SIZE = 64
image_directory = 'datasets/'


In [8]:
no_tumor_images = os.listdir(image_directory + 'no/')
yes_tumor_images = os.listdir(image_directory + 'yes/')


In [9]:
dataset = []
label = []


In [10]:
for i, image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'no/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)


In [11]:
for i, image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'yes/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)


In [12]:
print(len(dataset))
print(len(label))


3000
3000


In [13]:
dataset = np.array(dataset)
label = np.array(label)


In [14]:
x_train, x_test, y_train, y_test = train_test_split(
    dataset, label, test_size=0.2, random_state=42)

In [15]:
print(x_train.shape)
print(x_test.shape)
# (images, height, width, channels) 

(2400, 64, 64, 3)
(600, 64, 64, 3)


In [16]:
print(y_train.shape)
print(y_test.shape)


(2400,)
(600,)


In [17]:
# NORMALIZE THE DATA

x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)


In [42]:
# BUILD THE MODEL
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [43]:
# Binary Cross Entropy = 1, sigmoid activation function
# Categorical Cross Entropy = 2, softmax activation function

hs = model.compile(loss='binary_crossentropy',
                   optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=16, verbose=1, epochs=16,
          validation_data=(x_test, y_test), shuffle=False)


Epoch 1/16
150/150 [==============================] - 8s 51ms/step - loss: 0.5824 - accuracy: 0.6850 - val_loss: 0.4761 - val_accuracy: 0.7917
Epoch 2/16
150/150 [==============================] - 8s 52ms/step - loss: 0.4160 - accuracy: 0.8204 - val_loss: 0.3643 - val_accuracy: 0.8383
Epoch 3/16
150/150 [==============================] - 7s 47ms/step - loss: 0.3190 - accuracy: 0.8704 - val_loss: 0.3003 - val_accuracy: 0.8733
Epoch 4/16
150/150 [==============================] - 7s 48ms/step - loss: 0.2287 - accuracy: 0.9129 - val_loss: 0.2141 - val_accuracy: 0.9350
Epoch 5/16
150/150 [==============================] - 7s 47ms/step - loss: 0.1675 - accuracy: 0.9396 - val_loss: 0.1657 - val_accuracy: 0.9483
Epoch 6/16
150/150 [==============================] - 7s 46ms/step - loss: 0.1128 - accuracy: 0.9608 - val_loss: 0.1410 - val_accuracy: 0.9550
Epoch 7/16
150/150 [==============================] - 7s 47ms/step - loss: 0.0727 - accuracy: 0.9750 - val_loss: 0.1357 - val_accuracy: 0.9600

In [44]:
history.history

{'loss': [0.5824344158172607,
  0.41602975130081177,
  0.31898489594459534,
  0.22869931161403656,
  0.16754232347011566,
  0.11278015375137329,
  0.07268679887056351,
  0.054028384387493134,
  0.03399832919239998,
  0.018558531999588013,
  0.008017566055059433,
  0.0034945611841976643,
  0.004019171930849552,
  0.0021434216760098934,
  0.006406016182154417,
  0.008783207274973392],
 'accuracy': [0.6850000023841858,
  0.8204166889190674,
  0.8704166412353516,
  0.9129166603088379,
  0.9395833611488342,
  0.9608333110809326,
  0.9750000238418579,
  0.9833333492279053,
  0.9900000095367432,
  0.9954166412353516,
  0.9991666674613953,
  1.0,
  0.9991666674613953,
  1.0,
  0.9983333349227905,
  0.9975000023841858],
 'val_loss': [0.4760873019695282,
  0.36427852511405945,
  0.300307959318161,
  0.21412748098373413,
  0.16572001576423645,
  0.1409955620765686,
  0.13573449850082397,
  0.1459590494632721,
  0.12708719074726105,
  0.13169656693935394,
  0.17200854420661926,
  0.169622987508773

In [45]:
history.history['accuracy'][0]

0.6850000023841858

In [46]:
history.history['loss'][0]


0.5824344158172607

In [47]:
history.history['val_accuracy'][0]


0.7916666865348816

In [48]:
history.history['val_loss'][0]


0.4760873019695282

In [49]:
# model.save('brain_tumor_16epochs.h5')

In [50]:
y_pred = model.evaluate(x_test, y_test)

19/19 [==============================] - 0s 14ms/step - loss: 0.1802 - accuracy: 0.9650
